In [57]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import os
import cv2
import pandas as pd
from skimage import io
from skimage import color, exposure, transform
from PIL import Image, ImageChops, ImageDraw, ImageOps, ImageFilter 
from osgeo import gdal
from scipy import misc
import imutils
import argparse

In [58]:
def load_templates(directory):
    paths = []
    for files in os.listdir(directory):
        if (files != ".DS_Store"):
            paths.append(directory+'/'+files)
    return paths

In [65]:
def manipulate_images(paths):
    for image_path in paths:
        
        
        
        img = Image.open(image_path)
        pix = img.load()
        img.show()
        print "Image width: "+str(img.size[0])
        print "Image height: "+str(img.size[1])
        
        borders = []
        
        for y in range(0,img.size[1]):
            
            found = False
            
            for x in range(1,img.size[0]-1):
                r = pix[x,y][0]
                g = pix[x,y][1]
                b = pix[x,y][2]
                
                prev_r = pix[x-1,y][0]
                prev_g = pix[x-1,y][1]
                prev_b = pix[x-1,y][2]
                
                if ((abs(prev_r-r)>=20) or (abs(prev_g-g)>=20) or (abs(prev_b-b)>=20)):
                    for i in range(0,x):
                        borders.append([i,y])
                    found = True
                    break
            
            
            
            if (found == False):
                for i in range(0,img.size[0]-1):
                    borders.append([i,y])
            
              
            for x in range(img.size[0]-1,1,-1):
                
                r = pix[x,y][0]
                g = pix[x,y][1]
                b = pix[x,y][2]
                
                prev_r = pix[x-1,y][0]
                prev_g = pix[x-1,y][1]
                prev_b = pix[x-1,y][2]
                
                if ((abs(prev_r-r)>=20) or (abs(prev_g-g)>=20) or (abs(prev_b-b)>=20)):
                    for i in range(x,img.size[0]-1):
                        borders.append([i,y])
                    break
            
        
        for border in borders:
            pix[border[0],border[1]]= (0, 255, 0, 0) 
         
            
        img.show()
        
        
                    
        break
        

In [66]:
directory = 'Traffic_Signs_Templates/Images'
paths = load_templates(directory)
manipulate_images(paths)

Image width: 450
Image height: 446
